In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import time

import numpyro
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
from jax import lax

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

C:\Users\jakob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("Data/Data_processed.csv", sep=';')


for country in data['Country'].unique():
    d = data[data['Country'] == country]
    # sort by year
    d = d.sort_values(by='Year')
    # transform data to look at differences from year to year for all nummerical features (2 first columns are country and continent)
    d.iloc[:,5:] = d.iloc[:,5:].diff()
    # replace invividual nan values with 0
    data[data['Country'] == country] = d

data['Life expectancy difference'] = 0
for country in data['Country'].unique():
    d = data[data['Country'] == country]
    # sort by year
    d = d.sort_values(by='Year')
    # get difference in life expectancy
    d['Life expectancy difference'] = d['Life expectancy '].diff()


    # replace old values with new values
    data[data['Country'] == country] = d


In [3]:
data['Life expectancy difference 1 year ago'] = 0
data['Life expectancy difference 2 years ago'] = 0
# get difference in life expectancy for each country and year
for country in data['Country'].unique():
    d = data[data['Country'] == country]
    # sort by year
    d = d.sort_values(by='Year')
    # get difference in life expectancy
    d['Life expectancy difference'] = d['Life expectancy difference'].shift(-1)
    d['Life expectancy difference 1 year ago'] = d['Life expectancy difference'].shift(1)
    d['Life expectancy difference 2 years ago'] = d['Life expectancy difference'].shift(2)

    # replace old values with new values
    data[data['Country'] == country] = d

# remove year 2001 and 2002
# data = data[data['Year'] != 2001]

# loop over each year and print how many nan values there are
# for year in data['Year'].unique():
#     print(year, data[data['Year'] == year].isna().sum().sum())

# print average over contries life expectancy difference for each year
data = data.dropna()

In [5]:
train_split = data[data['Year'] < 2009]
test_split = data[data['Year'] >= 2009]

# make list of all columns that need to be standardized
cols = train_split.columns
# cols = cols.drop(['Country', 'continent', 'Year', 'Status'])
# standardize all columns except country, continent, year and status
for col in cols:
    if col not in ['Country', 'continent', 'Year', 'Status']:
        mean = train_split[col].mean()
        std = train_split[col].std()
        train_split[col] = (train_split[col] - mean) / std
        test_split[col] = (test_split[col] - mean) / std

# make numpy array of standardized data, first dimension is time, second is countries, third is features
countries_train = train_split['Country'].unique()
years_train = train_split['Year'].unique()
countries_test = test_split['Country'].unique()
years_test = test_split['Year'].unique()
cols = cols.drop(['Life expectancy ', 'Country', 'Year', 'continent', 'Life expectancy difference 1 year ago', 'Life expectancy difference 2 years ago', 'Life expectancy difference'])
train_split_np_features = np.empty((len(years_train), len(countries_train), len(cols)))
train_split_np_target = np.empty((len(years_train), len(countries_train)))
test_split_np_features = np.empty((len(years_test), len(countries_test), len(cols)))
test_split_np_target = np.empty((len(years_test), len(countries_test)))

target = 'Life expectancy difference'
for i, country in enumerate(countries_train):
    for j, year in enumerate(years_train):
        # take nummeric data of country and put in numpy array
        train_split_np_features[j, i] = train_split.loc[(train_split['Country'] == country) & (train_split['Year'] == year)][cols].to_numpy()
        # take target of country and put in numpy array
        train_split_np_target[j, i] = train_split.loc[(train_split['Country'] == country) & (train_split['Year'] == year)][target].to_numpy()
for i, country in enumerate(countries_test):
    for j, year in enumerate(years_test):
        # take nummeric data of country and put in numpy array
        test_split_np_features[j, i] = test_split.loc[(test_split['Country'] == country) & (test_split['Year'] == year)][cols].to_numpy()
        # take target of country and put in numpy array
        test_split_np_target[j, i] = test_split.loc[(test_split['Country'] == country) & (test_split['Year'] == year)][target].to_numpy()

test_split_np_features = np.concatenate((train_split_np_features[-2:], test_split_np_features))
test_split_np_target = np.concatenate((train_split_np_target[-2:], test_split_np_target))


C:\Users\jakob\AppData\Local\Temp\ipykernel_8740\1311968953.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_split[col] = (train_split[col] - mean) / std
C:\Users\jakob\AppData\Local\Temp\ipykernel_8740\1311968953.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_split[col] = (test_split[col] - mean) / std


In [78]:
def f(carry, h_external_change):
  beta2, z_prev1, z_prev2 = carry
  z_t = beta2[0]*z_prev1 + beta2[1]*z_prev2 + h_external_change
  z_prev2 = z_prev1
  z_prev1 = z_t
  return (beta2, z_prev1, z_prev2), z_t


def model(obs_x, obs_y):
    time = obs_x.shape[0]
    countries = obs_x.shape[1]
    features_x = obs_x.shape[2]
    

    beta1 = numpyro.sample(name="beta1", fn=dist.Normal(loc=0., scale=1), sample_shape=(features_x,))
    xs = numpyro.sample(name="xs", fn=dist.Normal(loc=0., scale=1.), sample_shape=(time, countries, features_x, ), obs=obs_x)
    beta2 = numpyro.sample(name="beta2", fn=dist.Normal(loc=0., scale=1), sample_shape=(2,))
    tau = numpyro.sample(name="tau", fn=dist.HalfCauchy(scale=.1))
    sigma = numpyro.sample(name="sigma", fn=dist.HalfCauchy(scale=.1))
    z0 = numpyro.sample(name="z0", fn=dist.Normal(loc=0, scale=.1), sample_shape=(countries,))
    z1 = numpyro.sample(name="z1", fn=dist.Normal(loc=0, scale=.1), sample_shape=(countries,))

    h_external_change = numpyro.sample(name="h_change", fn=dist.Normal(loc=xs @ beta1, scale=tau))

    carry = (beta2, z0, z1)
    z_collection = [z0, z1]
    carry, zs_exp = lax.scan(f, carry, h_external_change, time)
    z_collection = jnp.concatenate((jnp.array(z_collection), zs_exp), axis=0)

    numpyro.sample(name="y_obs", fn=dist.Normal(loc=zs_exp, scale=sigma), obs=obs_y)
    return z_collection

In [79]:
print(train_split_np_features.shape, test_split_np_features.shape, train_split_np_target.shape, test_split_np_target.shape)

t0 = time.time()

rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

nuts_kernel = NUTS(model=model, max_tree_depth=8, step_size=5e-3, adapt_step_size=False)
mcmc = MCMC(nuts_kernel, num_samples=100, num_warmup=50, num_chains=1, progress_bar=True)
mcmc.run(rng_key_, obs_x=train_split_np_features, obs_y=train_split_np_target)

t_fin = time.time()

print("Total time: {0:.3f}m".format((t_fin - t0)/60))

(7, 183, 16) (8, 183, 16) (7, 183) (8, 183)


TypeError: '>' not supported between instances of 'tuple' and 'int'

In [10]:
beta1 = mcmc.get_samples()['beta1'].mean(0)
beta2 = mcmc.get_samples()['beta2'].mean(0)
errors = []
errors_baseline = []
for i in range(2, test_split_np_features.shape[0]):
    prediction = beta2[0] * test_split_np_target[i-1] + beta2[1] * test_split_np_target[i-2] + test_split_np_features[i] @ beta1
    errors.append(np.mean((prediction - test_split_np_target[i])**2))
    errors_baseline.append(np.mean(test_split_np_target[i]**2))
print(np.mean(errors))
print(np.mean(errors_baseline))

2.1021454
2.1955322823340517
